# 个性化电商广告推荐系统业务分析与实现

## 一、基础推荐系统业务流程/技术架构分析

**推荐系统基本组成：**

    - 基础数据层：基础数据收集和存储，为数据处理层提供基础数据
    - 数据处理层：数据预处理、特征工程、数据分析运算，为推荐业务层提供缓存数据
    - 推荐业务层：处理推荐请求，建立推荐任务，根据召回的候选集合、排序、过滤的实现，完成推荐任务生成最终推荐结果

**基础推荐系统业务流程：**
![image.png](images/RS基础业务架构.png)

#### 推荐系统主要流程： 

    - 基础数据层：
        - 业务数据库 ==> 用户/物品基本属性信息数据 ==> 分布式文件系统 >>>> offline
        
        - 日志系统 ==> 推荐系统业务日志数据 ==> 分布式文件系统 >>>> offline
        
        - 日志系统 ==> 推荐系统业务日志数据 ==> 消息队列 >>>>  online
    
    - 数据处理层：
    
        - offline(离线处理/批量处理)：T+1的推荐数据更新(求准)
        
            - 用户历史行为日志数据  ==> 评分规则(自定义)  ==> 用户-物品偏好评分 ==> 评分矩阵/ALS模型 >>>> 协同过滤离线召回
            
            - 用户历史行为日志数据 ==> 历史样本数据  >>>> 排序模型训练
            
            - 用户历史行为日志数据 + 用户/物品基本属性信息数据 ==> 兴趣特征 + 属性特征 + 画像特征
            
            - 兴趣特征（仅样本） + 属性特征（仅样本）  ==> 样本特征数据  >>>> 排序模型训练
            
            - (历史样本数据 + 样本特征数据) + 逻辑回归算法(LR)/梯度提升决策树算法(GBDT) ==> 排序模型  >>>> 在线实时排序
            
            - 兴趣特征（所有） + 属性特征（所有） ==> 缓存(离线特征) >>>  在线实时排序
            
        - online(在线处理/流式处理)：秒级推荐数据更新(求快)
        
            - 用户实时行为日志数据 ==> 提取属性信息特征 ==> 缓存(实时特征)
            
            - 用户实时行为日志数据 ==> 提取关联特征 ==> 实时召回 ==> 缓存(实时召回集)
            
            - 实时特征/实时召回？
    
    - 推荐业务层：召回 ===> 排序 ===> 过滤
    
        - offline：进行离线召回产生候选集并进行缓存
        
            - 离线召回(协同过滤召回 + 画像召回 + 热门召回) ==> 召回候选集 ==> 缓存(离线召回集)
            
        - online(推荐任务处理)：实时推荐
        
            - 处理推荐请求 ==> 建立推荐任务(用户id，场景id) ==> 排序模型 + 特征数据(来自缓存) ==> 排序结果 ==> 过滤 ==> 最终推荐结果

2. 基于的Lambda架构的基础推荐系统技术架构

![image.png](images/RS基础技术架构.png)

## 二、电商广告推荐

电商广告推荐通常使用广告点击率(CTR--Click-Through-Rate)预测来实现

#### 点击率预测 VS 推荐算法

点击率预测需要给出精准的点击概率，比如广告A点击率0.5%、广告B的点击率0.12%等；而推荐算法很多时候只需要得出一个最优的次序A>B>C即可。

点击率预测使用的算法通常是如逻辑回归(Logic Regression)这样的机器学习算法，而推荐算法则是一些基于协同过滤推荐、基于内容的推荐等思想实现的算法

#### 点击率 VS 转化率

点击率预测是对每次广告的点击情况做出预测，可以判定这次为点击或不点击，也可以给出点击或不点击的概率

转化率指的是从状态A进入到状态B的概率，电商的转化率通常是指到达网站后，进而有成交记录的用户比率，如用户成交量/用户访问量

#### 搜索和非搜索广告点击率预测的区别

搜索中有很强的搜索信号-“查询词(Query)”，查询词和广告内容的匹配程度很大程度影响了点击概率，搜索广告的点击率普遍较高

非搜索广告（例如展示广告，信息流广告）的点击率的计算很多就来源于用户的兴趣和广告自身的特征，以及上下文环境。通常好位置能达到百分之几的点击率。对于很多底部的广告，点击率非常低，常常是千分之几，甚至更低

## 三、数据集介绍

Ali_Display_Ad_Click是阿里巴巴提供的一个淘宝展示广告点击率预估数据集

数据集来源：天池竞赛

#### 1. 原始样本骨架raw_sample

淘宝网站中随机抽样了114万用户8天内的广告展示/点击日志（2600万条记录），构成原始的样本骨架。
字段说明如下： 
1. user_id：脱敏过的用户ID；
2. adgroup_id：脱敏过的广告单元ID；
3. time_stamp：时间戳；
4. pid：资源位；
5. noclk：为1代表没有点击；为0代表点击；
6. clk：为0代表没有点击；为1代表点击；

用前面7天的做训练样本（20170506-20170512），用第8天的做测试样本（20170513）

#### 2. 广告基本信息表ad_feature
本数据集涵盖了raw_sample中全部广告的基本信息(约80万条目)。字段说明如下： 
1. adgroup_id：脱敏过的广告ID；
2. cate_id：脱敏过的商品类目ID；
1. campaign_id：脱敏过的广告计划ID；
1. customer_id:脱敏过的广告主ID；
1. brand_id：脱敏过的品牌ID；
1. price: 宝贝的价格

其中一个广告ID对应一个商品（宝贝），一个宝贝属于一个类目，一个宝贝属于一个品牌。

#### 3. 用户基本信息表user_profile
本数据集涵盖了raw_sample中全部用户的基本信息(约100多万用户)。字段说明如下： 
1. userid：脱敏过的用户ID；
1. cms_segid：微群ID；
1. cms_group_id：cms_group_id；
1. final_gender_code：性别 1:男,2:女；
1. age_level：年龄层次；  1234
1. pvalue_level：消费档次，1:低档，2:中档，3:高档；
1. shopping_level：购物深度，1:浅层用户,2:中度用户,3:深度用户
1. occupation：是否大学生 ，1:是,0:否
1. new_user_class_level：城市层级

#### 4. 用户的行为日志behavior_log
本数据集涵盖了raw_sample中全部用户22天内的购物行为(共七亿条记录)。字段说明如下：
1. user：脱敏过的用户ID；
1. time_stamp：时间戳；
1. btag：行为类型,  包括以下四种：
    - 类型 | 说明
    - pv |	浏览
    - cart |	加入购物车
    - fav | 喜欢
    - buy | 购买
1. cate_id：脱敏过的商品类目id；
1. brand_id: 脱敏过的品牌id；

这里以user + time_stamp为key，会有很多重复的记录；这是因为我们的不同的类型的行为数据是不同部门记录的，在打包到一起的时候，实际上会有小的偏差（即两个一样的time_stamp实际上是差异比较小的两个时间）

In [1]:
!du -h /root/notebook_dir/project1-ad-rs/datasets/*

9.6M	/root/notebook_dir/project1-ad-rs/datasets/ad_feature.csv.tar.gz
4.7G	/root/notebook_dir/project1-ad-rs/datasets/behavior_log.csv.tar.gz
232M	/root/notebook_dir/project1-ad-rs/datasets/raw_sample.csv.tar.gz
5.9M	/root/notebook_dir/project1-ad-rs/datasets/user_profile.csv.tar.gz


## 四、 电商广告推荐分析

主要包括

- 一份广告点击的样本数据raw_sample.csv：体现的是用户对不同位置广告点击、没点击的情况

- 一份广告基本信息数据ad_feature.csv：体现的是每个广告的类目(id)、品牌(id)、价格特征

- 一份用户基本信息数据user_profile.csv：体现的是用户群组、性别、年龄、消费购物档次、所在城市级别等特征

- 一份用户行为日志数据behavior_log.csv：体现用户对商品类目(id)、品牌(id)的浏览、加购物车、收藏、购买等信息

我们是在对非搜索类型的广告进行点击率预测和推荐(没有搜索词、没有广告的内容特征信息)

1. 推荐业务处理主要流程： 召回 ===> 排序 ===> 过滤

    - 离线处理业务流
    
        - raw_sample.csv ==> 历史样本数据
        
        - ad_feature.csv ==> 广告特征数据
        
        - user_profile.csv ==> 用户特征数据
        
        - raw_sample.csv + ad_feature.csv + user_profile.csv ==> CTR点击率预测模型
        
        - behavior_log.csv ==> 评分数据 ==> user-cate/brand评分数据 ==> 协同过滤 ==> top-N cate/brand ==> 关联广告
        
        - 协同过滤召回 ==> top-N cate/brand ==> 关联对应的广告完成召回

    - 在线处理业务流
    
        - 数据处理部分：
        
            - 实时行为日志 ==>  实时特征  ==> 缓存
            
            - 实时行为日志 ==>  实时商品类别/品牌 ==> 实时广告召回集 ==> 缓存
            
        - 推荐任务部分：
        
            - CTR点击率预测模型 + 广告/用户特征(缓存) + 对应的召回集(缓存) ==> 点击率排序 ==> top-N 广告推荐结果

2. 涉及技术：Flume、Kafka、Spark-streming\HDFS、Spark SQL、Spark ML、Redis

    - Flume：日志数据收集
    
    - Kafka：实时日志数据处理队列
    
    - HDFS：存储数据
    
    - Spark SQL：离线处理
    
    - Spark ML：模型训练
    
    - Redis：缓存

